In [58]:
from TSPGraph import Graph
import numpy as np
from numba import jit
from numba import njit, prange
import copy
from tqdm import tqdm
from multiprocessing import Pool
import time
from copy import deepcopy
from functools import lru_cache

In [2]:
graph = Graph('C')
obj = graph.objective
dist = graph.dist_matrix

In [3]:
@jit
def random_solution():
    sol = np.random.choice(np.arange(200),size=100,replace=False)
    return np.append(sol,sol[0])

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/3766937091.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def random_solution():


In [4]:
combination_pairs = [(x,y) for x in range(1,100) for y in range(x+2, 100)]
combination_pairs_edges = [(x,y) for x in range(1,100) for y in range(x+2, 100)]

In [5]:
@jit
def ExchangeVertices(solution, i, j):
    result = copy.deepcopy(solution)
    result[[i,j]] = result[[j,i]]
    return result
@jit
def ExchangeVerticesDelta(dist, solution, i, j):
    res = (dist[solution[i-1], solution[j]] + dist[solution[j], solution[i+1]] + dist[solution[j-1], solution[i]] + dist[solution[i], solution[j+1]]) - (dist[solution[i-1], solution[i]] + dist[solution[i], solution[i+1]] + dist[solution[j-1], solution[j]] + dist[solution[j], solution[j+1]])
    return res
@jit
def MinInAllExchangedVertices(dist, solution):
    minDelta = 0
    resultExchange = tuple()
    for x in combination_pairs:
        delta = ExchangeVerticesDelta(dist, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultExchange = (x[0], x[1])
    return [minDelta, resultExchange]
@jit
def GreedyExchangeVertex(dist, solution):
    for x in combination_pairs:
        delta = ExchangeVerticesDelta(dist, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None


MinInAllExchangedVertices(dist, random_solution())

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/2949073171.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def ExchangeVertices(solution, i, j):
/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/2949073171.py:7: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def ExchangeVerticesDelta(dist, solution, i, j):
/var/folders/vt/8kg_3

[-10145, (37, 51)]

In [6]:
@jit
def ExchangeEdges(solution, i, j):
    assert i<j
    result = copy.deepcopy(solution)
    result = np.concatenate((result[:i], result[j:i-1:-1], result[j+1:]))
    return result
@jit
def ExchangeEdgesDelta(dist, solution, i, j):
    assert i<j
    res = (dist[solution[i-1], solution[j]] + dist[solution[i], solution[j+1]]) - (dist[solution[i-1], solution[i]] + dist[solution[j], solution[j+1]])
    return res
@jit
def MinInAllExchangedEdges(dist, solution):
    minDelta = 0
    resultExchange = tuple()
    for x in combination_pairs_edges:
        delta = ExchangeEdgesDelta(dist, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultExchange = (x[0], x[1])
    return (minDelta, resultExchange)
@jit
def GreedyExchangeEdges(dist, solution):
    for x in combination_pairs_edges:
        delta = ExchangeEdgesDelta(dist, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None

MinInAllExchangedEdges(dist, random_solution())

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/432162731.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def ExchangeEdges(solution, i, j):
/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/432162731.py:8: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def ExchangeEdgesDelta(dist, solution, i, j):
/var/folders/vt/8kg_3c6554l1g

(-6192, (38, 42))

In [7]:
@jit
def SwapVertexWithNewOne(solution, i, a):
    result = copy.deepcopy(solution)
    result[i] = a
    return result
@jit
def SwapVertexWithNewOneDelta(obj, solution, i, a):
    return (obj[solution[i-1], a] + obj[a, solution[i+1]]) - (obj[solution[i-1], solution[i]] + obj[solution[i], solution[i+1]])
@jit
def MinInAllSwaps(solution, obj):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    #pairs of index in current cycle - node 
    pairs = np.transpose([np.tile(domain[:100], 100), np.repeat(not_in_sol, 100)])
    minDelta = 0
    resultSwap = tuple()
    for x in pairs:
        delta = SwapVertexWithNewOneDelta(obj, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultSwap = (x[0], x[1])
    return (minDelta, resultSwap)
@jit
def GreedySwap(solution, obj):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    #pairs of index in current cycle - node 
    pairs = np.transpose([np.tile(domain[:100], 100), np.repeat(not_in_sol, 100)])
    for x in pairs:
        delta = SwapVertexWithNewOneDelta(obj, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None
print(MinInAllSwaps(random_solution(), obj))

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/3288179290.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def SwapVertexWithNewOne(solution, i, a):
/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/3288179290.py:7: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def SwapVertexWithNewOneDelta(obj, solution, i, a):
/var/folders/v

(-6566, (17, 96))


In [16]:
@jit
def SteepestLocalSearch(obj, dist, initial_solution, type, graph):
    current_solution = initial_solution
    max_iter = 1_000_000
    while(max_iter > 0):
        # print(current_cost)
        best_swap_val, best_swap = MinInAllSwaps(current_solution, obj)
        if type == 'edge':
            best_exchange_val, best_exchange = MinInAllExchangedEdges(dist, current_solution)
        else:
            best_exchange_val, best_exchange = MinInAllExchangedVertices(dist, current_solution)
        
        if best_swap_val >= 0 and best_exchange_val >= 0:
            break

        if best_swap_val < best_exchange_val:
            current_solution = SwapVertexWithNewOne(current_solution, best_swap[0], best_swap[1])
            # print('swap' + str(best_swap_val))
        elif type == 'edge':
            current_solution = ExchangeEdges(current_solution, best_exchange[0], best_exchange[1])
            # print('exchange ' + str(best_exchange_val) + str(best_exchange))
        else:
            current_solution = ExchangeVertices(current_solution, best_exchange[0], best_exchange[1])
            # print('exchange ' + str(best_exchange_val) + str(best_exchange))

        max_iter -= 1
    return current_solution, graph.cycle_cost(current_solution)
        
SteepestLocalSearch(obj, dist, random_solution(), 'edge', graph)

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/4110361867.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def SteepestLocalSearch(obj, dist, initial_solution, type, graph):
/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/4110361867.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "SteepestLocalSearch" failed type inference due to: non-precise type pyobject
During: typing of argument at /var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/4110361867.py (1)

File "../../../../../../var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/4110361867.py", line 1:
<

(array([ 32,  62,  53,  22,  55,  30, 195,  74, 163, 113, 181,  25,  85,
         36, 132, 128, 118,  84,  37, 158, 185,  64,  27, 147,  96,  59,
        159, 143,  34, 164, 178,  19,  69,   0, 149,  50, 121,  91,   4,
        114,  77, 192, 199, 174, 137, 177,  41,   1, 152,  11, 160, 106,
         48,  92,  26, 119, 130,  75, 189, 109, 134,  95, 110, 169,   8,
        124,  80,  31,  89,  94,  72, 112,  51, 135,  99, 101,  60,   9,
        175, 167,  88, 153, 127, 186,  45,  98,  66, 156, 172,   6, 141,
         87, 144,  79, 194,  21, 171, 117,  15, 108,  32]),
 52156)

In [59]:
@lru_cache(maxsize=1_000_000)
def addition_cost(graph, triplet):
    return graph.objective[triplet[0], triplet[1]] + graph.objective[triplet[1], triplet[2]] - graph.objective[triplet[0], triplet[2]]

In [85]:
@jit
def getRegret2Weighted(graph, cycle):
    candidates = np.delete(np.arange(200), cycle)
    # print(candidates)
    costs = [-9999999] * 200
    solutions = [0] * 200
    weigth = 0.61
    scores = [-9999999] * 200
    temporaryCost = graph.cycle_cost(cycle)
    for candidate in candidates:
        currentCosts = []
        currentSolutions = []
        for i in range(len(cycle) - 1):
            currentCosts.append(graph.cycle_cost(cycle) + addition_cost(graph,(cycle[i],candidate,cycle[i+1])))
            currentSolutions.append(np.concatenate((cycle[:i + 1],np.array([candidate]).reshape((1,)), cycle[i + 1:])))
        firstBestId = np.argmin(currentCosts)
        firstBestCost = currentCosts[firstBestId]
        currentCosts = np.delete(currentCosts, firstBestId)
        secondBestId = np.argmin(currentCosts)
        secondBestCost = currentCosts[secondBestId]
        bestSolution = currentSolutions[firstBestId]

        newTemporaryCost = graph.cycle_cost(bestSolution)
        difference = newTemporaryCost - temporaryCost
        scores[candidate] = weigth * (secondBestCost - firstBestCost) - (1 - weigth) * difference
        costs[candidate] = newTemporaryCost
        solutions[candidate] = bestSolution

    bestRegretId = np.argmax(scores)
    cycle = solutions[bestRegretId]
    temporaryCost = costs[bestRegretId]

    # print(cycle)
    return cycle

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/689422102.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def getRegret2Weighted(graph, cycle):


In [84]:
# @jit
# def repairCycle(graph, cycle):
#   while -1 in cycle:
#     print(np.count_nonzero(cycle==-1))
#     cycle = getRegret2Weighted(graph, cycle)

#   return cycle
@jit
def repairCycle(graph, cycle):
  cycle = [node for node in cycle if node != -1]

  while(len(cycle) < 101):
    cycle = getRegret2Weighted(graph, cycle)

  return cycle

/var/folders/vt/8kg_3c6554l1gb7hr4vbyjgr0000gn/T/ipykernel_30810/1305367034.py:9: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def repairCycle(graph, cycle):


In [117]:
def createOffspringOperator1(firstParent, secondParent, graph):
  offspring = np.array([-1 for i in range(101)])
  
  for i in range(len(firstParent)):
    if firstParent[i] in secondParent:
      offspring[i] = firstParent[i]

  for i in range(101):
    potentialNodes = list(set(range(200)) - set(offspring))

    if offspring[i] == - 1:
      offspring[i] = np.random.choice(potentialNodes, 1)[0]

  return offspring, graph.cycle_cost(offspring)

In [118]:
def createOffspringOperator2(firstParent, secondParent, graph):
  offspring = np.array([-1 for _ in range(101)])

  for i in range(len(firstParent)):
    if firstParent[i] in secondParent:
      offspring[i] = firstParent[i]

  offspring = repairCycle(graph, offspring)
  return offspring, graph.cycle_cost(offspring)

In [109]:
def createOffsprings(firstParent, secondParent, graph):
  firstOffspring = createOffspringOperator1(firstParent, secondParent, graph)
  secondOffspring = createOffspringOperator2(firstParent, secondParent, graph)

  return firstOffspring, secondOffspring


In [139]:
def createNewPopulation(population, graph):
  childrenSize = 10
  newPopulation = deepcopy(population)

  for _ in range(childrenSize):
    firstParentId, secondParentId = np.random.choice(len(population), 2, replace=False)
    firstParent, secondParent = population[firstParentId][0], population[secondParentId][0]

    firstOffspring, secondOffspring = createOffsprings(firstParent, secondParent, graph)

    populationCosts = [solution[1] for solution in newPopulation]

    # checking costs as identifier of the solution, since comparing array returned
    # strange runtime errors
    if not firstOffspring[1] in populationCosts:
      newPopulation.append(firstOffspring)

    if not secondOffspring[1] in populationCosts:
      newPopulation.append(secondOffspring)
  
  return sorted(newPopulation, key = lambda solution: solution[1])[:len(population)]

In [134]:
def hybridEvolutionaryAlgorithm(obj, dist, timeLimit, graph):
  population = [SteepestLocalSearch(obj, dist, random_solution(), 'edge', graph) for _ in range(20)]
  bestSolution = population[0]
  timeout = time.time() + timeLimit
  while(time.time() < timeout):
      population = createNewPopulation(population, graph)

      if population[0][1] < bestSolution[1]:
         bestSolution = population[0]

    
  return bestSolution[0], bestSolution[1]

In [142]:
def hybridEvolutionaryAlgorithmExperiment(obj, dist, graph):
  solutions = []
  costs = []
  for i in range(20):
    bestSolution, bestCost = hybridEvolutionaryAlgorithm(obj, dist, 140, graph)
    solutions.append(bestSolution)
    costs.append(bestCost)
    print(bestCost)
  return solutions, costs

In [143]:
graph = Graph('C')

solutionsC, costsC = hybridEvolutionaryAlgorithmExperiment(graph.objective, graph.dist_matrix, graph)

/opt/homebrew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


48512
48970
48720
49303
49556
48385
49586
49224
49015
48641
49025
49305
48283
49089
49389
48558
48661
49264
48910
48630


In [144]:
print(np.min(costsC), np.max(costsC), np.mean(costsC), np.median(costsC))

48283 49586 48951.3 48992.5


In [145]:
graph = Graph('D')

solutionsD, costsD = hybridEvolutionaryAlgorithmExperiment(graph.objective, graph.dist_matrix, graph)

/opt/homebrew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


45530
44460
45548
44998
46272
45389
46058
45221
45991
44857
45477
45090
45369
44883
45266
46118
46045
45826
44438
45891


In [146]:
print(np.min(costsD), np.max(costsD), np.mean(costsD), np.median(costsD))

44438 46272 45436.35 45433.0
